In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])

df_train.head()

In [ ]:
# let us perform a linear regression - start with a few features 

In [ ]:
def feature_summary(data):
    n_row=data.shape[0]
    features=pd.DataFrame()
    features_names=[]
    features_type = []
    features_counts=[]
    features_missing=[]
    names=data.columns.values
    for i in names:
        features_names.append(i)
        features_type.append(type(data.ix[1,i]))
        features_counts.append(data[i].value_counts().count())
        features_missing.append(data[data[i].isnull()].shape[0])
    features['name']=features_names
    features['type'] = features_type
    features['value counts']=features_counts
    features['missing']=features_missing
    features['percentage_missing']=features['missing']/n_row
    return (features)

In [ ]:
feature_summ = feature_summary(df_train)

In [ ]:
column_names = feature_summ[feature_summ['percentage_missing'] < 0.20]

In [ ]:
column_names

In [ ]:
df_train_quant = df_train.select_dtypes(include=['int64', 'floating'])

In [ ]:
df_train_quant.head()

In [ ]:
corr_mat = pd.DataFrame()
column_names = np.arange(0)
correlation_values = np.arange(0)
for i in df_train_quant.columns.values:
    column_names = np.append(column_names, i)
    correlation_values = np.append(correlation_values,df_train_quant[i].corr(df_train_quant['price_doc']))

In [ ]:
corr_mat['columns'] = column_names
corr_mat['value'] = correlation_values

In [ ]:
corr_mat

In [ ]:
corr_mat.plot(kind = 'bar')

In [ ]:
# using the above poorly constructed plot, and if you look closely you'll see why i chose 0.20
# as a theshold
corr_mat_select = corr_mat[corr_mat['value'].abs() > 0.20 ]

In [ ]:
corr_mat_select.shape

In [ ]:
# Implementing Linear Regression

In [ ]:
# let us use linear regression to solve this
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm 

In [ ]:
df_train_quant = df_train[corr_mat_select['columns']]

In [ ]:
df_train_quant.head()

In [ ]:
#handling missing values using mean values? we can explore more methods for this at a later
#stage
#df_train_quant = df_train_quant.drop('timestamp', 1)
for i in range(1,len(df_train_quant.columns)):
       df_train_quant.iloc[:,i] = df_train_quant.iloc[:,i].fillna(df_train_quant.iloc[:,i].mean())

In [ ]:
y = df_train_quant['price_doc']
df_train_quant = df_train_quant.drop('price_doc', 1)
X_train,X_test,y_train,y_test = train_test_split(df_train_quant,y,test_size=0.20)

In [ ]:
model = lm.LinearRegression()
model.fit(X_train, y_train)

In [ ]:
predy = model.predict(X_train)

In [ ]:
residuals = (predy - y_train) * (predy - y_train)

In [ ]:
import matplotlib.pyplot as plt
model_plot = pd.DataFrame()
model_plot['y'] = y_train
model_plot['residuals'] = residuals
plt.plot(y_train, residuals)

In [ ]:
# We can see that the errors start increasing after the value is 17500. Maybe
# we can set this as a threshold.

In [ ]:
predy = model.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test,predy)
corr_mat_select = corr_mat_select.drop(275)
test = df_test[corr_mat_select['columns']]
for i in range(1,len(test.columns)):
       test.iloc[:,i] = test.iloc[:,i].fillna(test.iloc[:,i].mean())
result = model.predict(test)
result = pd.DataFrame(result)
result['id'] = df_test['id']
result['price_doc'] = result.iloc[:,0].abs()
result = result.drop(0,1)
result['price_doc'] = result['price_doc'].round(2)
result.to_csv("output_10.csv", index = False)

In [ ]:
# The above error plot gives us some insight. 
# The increasing value of error as y value increases shows us that we need to transform y values
# or break it down.


In [ ]:
# Let us separate our models by breaking our dataframes based on y values on the set threshold

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

    # Input data files are available in the "../input/" directory.
    # For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm 
from sklearn.metrics import r2_score
# Any results you write to the current directory are saved as output.

df_train = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])

def feature_summary(data):
    n_row=data.shape[0]
    features=pd.DataFrame()
    features_names=[]
    features_type = []
    features_counts=[]
    features_missing=[]
    names=data.columns.values
    for i in names:
        features_names.append(i)
        features_type.append(type(data.ix[1,i]))
        features_counts.append(data[i].value_counts().count())
        features_missing.append(data[data[i].isnull()].shape[0])
    features['name']=features_names
    features['type'] = features_type
    features['value counts']=features_counts
    features['missing']=features_missing
    features['percentage_missing']=features['missing']/n_row
    return (features)

In [ ]:
def normalize(test_series):
    normalized = np.log(test_series)
    return normalized
def denormalize(test_series):
    denormalized = np.exp(test_series)
    return denormalized

In [ ]:
y_log = normalize(df_train['price_doc'])

In [ ]:
#column_names = feature_summ[feature_summ['percentage_missing'] < 0.20]
df_train_quant = df_train.select_dtypes(include=['int64', 'floating'])
corr_mat = pd.DataFrame()
column_names = np.arange(0)
correlation_values = np.arange(0)
for i in df_train_quant.columns.values:
    column_names = np.append(column_names, i)
    correlation_values = np.append(correlation_values,df_train_quant[i].corr(df_train_quant['price_doc']))
corr_mat['columns'] = column_names
corr_mat['value'] = correlation_values

In [ ]:
# using the above poorly constructed plot, and if you look closely you'll see why i chose 0.20
# as a theshold
corr_mat_select = corr_mat[corr_mat['value'].abs() > 0.20 ]

In [ ]:
# let us use linear regression to solve this
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm 
df_train_quant = df_train[corr_mat_select['columns']]
#handling missing values using mean values? we can explore more methods for this at a later
#stage
#df_train_quant = df_train_quant.drop('timestamp', 1)
for i in range(1,len(df_train_quant.columns)):
       df_train_quant.iloc[:,i] = df_train_quant.iloc[:,i].fillna(df_train_quant.iloc[:,i].mean())
y_normalized = normalize(df_train_quant['price_doc'])
y_original = df_train_quant['price_doc']
df_train_quant = df_train_quant.drop('price_doc', 1)
X_train,X_test,y_train,y_test = train_test_split(df_train_quant,y_normalized,test_size=0.20)

In [ ]:
model = lm.LinearRegression()
model.fit(X_train, y_train)

In [ ]:
predy = model.predict(X_train)
from sklearn.metrics import r2_score
r2_score(denormalize(predy), denormalize(y_train))

In [ ]:
denormalize(predy)

In [ ]:
denormalize(y_train)